# Advanced MCMC FAQ

This FAQ will cover several "advanced" topics that might come up in the context of doing MCMC with ``py21cmmc``. Throughout we will use the following imports:

In [1]:
from py21cmmc import mcmc, __version__

print(__version__)

0.1.0


## Continue Running a Chain After it has Stopped?

``21CMMC`` has an inbuilt custom chain storage mechanism which uses ``h5py``. The storage class itself is found here:

In [2]:
from py21cmmc.mcmc.cosmoHammer.storage import HDFStorage

This class should not be required to be instantiated directly, however. In any case, this class has in-built mechanisms for detecting which iteration the chain is currently at, and saves the complete random state of the chain at each iteration. Thus, it can be started from wherever it stops (due to an exception, or walltime on a shared system). In practice, doing so is as easy as calling ``run_mcmc`` with the ``continue_sampling`` set to ``True`` (which it is by default). One just needs to ensure that the data directory and model name are consistent so that the previous chain can be located:

In [ ]:
mcmc.run_mcmc(
    core, likelihood,
    datadir='.', model_name='a_model',
    sampleIterations = 100
)

# ... process is stopped at 50 iterations...

mcmc.run_mcmc(
    core, likelihood,
    datadir='.', model_name='a_model',
    sampleIterations = 100,
    continue_sampling=True
)

# ... will complete the 100 iterations.

Note that continuing the sampling will only complete up to the number of iterations given, rather than *doing* that number of iterations on that particular call. 